# 509 Final Project

## Rsolve dependencies

## Globally import libraries

In [ ]:
import numpy as np
import pandas as pd
import pymysql as mysql
import matplotlib.pyplot as plt
import os
import shutil
import re
import logging
import time
import zipfile
import requests
from bs4 import BeautifulSoup
import datetime
import re
import regex as rex
from collections import defaultdict, Counter
import random
#import mysql.connector

# Set pandas global options
pd.options.display.max_rows = 17

## Initiate MySQL connection

In [ ]:
'''Set local environment variables to hide user name & password citation:
https://www.geeksforgeeks.org/how-to-hide-sensitive-credentials-using-python/'''

user_name = os.environ['MySQLUSRAC']
user_pass = os.environ['MySQLPWDAC']

# Instantiate connection
db_conn = mysql.connect(host='localhost',
                        port=int(3306),
                        user=user_name,
                        passwd=user_pass,
                        db='509_final_proj')

# Create a cursor object
cursor = db_conn.cursor()

In [ ]:
tbl_names = pd.read_sql('SHOW TABLES', db_conn)

display(tbl_names)
print(type(tbl_names))

### Establish logging policy

In [ ]:
'''Logging citations (see additional code in following code blocks:
OpenAI. (2021). ChatGPT [Computer software]. https://openai.com/;
https://docs.python.org/3/howto/logging.html#logging-basic-example;
https://docs.python.org/3/howto/logging.html#logging-to-a-file;
https://docs.python.org/3/howto/logging-cookbook.html#using-a-rotating-log-file-handler;
https://docs.python.org/3/howto/logging-cookbook.html#using-a-timed-rotating-file-handler'''

# Set up logging
logging.basicConfig(level=logging.INFO,
                    filename='pymysql.log',
                    filemode='a',
                    format='>>>>>>>>>>>>>><<<<<<<<<<<<<<\n%(asctime)s - %(levelname)s - %(message)s')

## Read URLs from MySQL table to perform web scraping

In [ ]:
'''Set local environment variables to hide user name & password citation:
https://www.geeksforgeeks.org/how-to-hide-sensitive-credentials-using-python/'''

user_name = os.environ['MySQLUSRAC']
user_pass = os.environ['MySQLPWDAC']

# Instantiate connection
db_conn = mysql.connect(host='localhost',
                        port=int(3306),
                        user=user_name,
                        passwd=user_pass,
                        db='509_final_proj')

# Create a cursor object
cursor = db_conn.cursor()

In [ ]:
nat_tbl_name = 'nar_temp'
nwa_tbl_name = 'news_articles'

'''Connect to MqSQL table in batches citation:
OpenAI'''
# Batch size (number of URLs to process at a time)
batch_size = 10

# Get the total number of URLs in the table
count_query = f"SELECT COUNT(*) FROM {nwa_tbl_name}"
cursor.execute(count_query)
total_urls = cursor.fetchone()[0]
print(f'URL Count: {total_urls}')

# Calculate the number of batches required
num_batches = (total_urls // batch_size) + 1

# Process URLs in batches
for batch in range(num_batches):
    offset = batch * batch_size

    # Retrieve URLs from the MySQL table in the current batch
    query = f"SELECT url FROM {nwa_tbl_name} LIMIT {batch_size} OFFSET {offset}"
    print(query)
    cursor.execute(query)
    urls = cursor.fetchall()
    print(f'URL batch: {urls}')

    # Iterate over the URLs and scrape their contents
    for url in urls:
        url = url[0]  # Extract the URL from the tuple

        # Make an HTTP request to the URL
        response = requests.get(url)
        time.sleep(5 + 10 * random.random())

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract the raw text from the HTML
            #print(soup.prettify())
            #raw_text = soup.get_text()
            raw_text = soup.prettify()


            # Update the MySQL table with the scraped text
            update_query = "UPDATE news_articles SET article_text = %s WHERE url = %s"
            print(update_query)
            cursor.execute(update_query, (raw_text, url))
            db_conn.commit()
        else:
            print(f'Response: {response.status_code}')

### Commit changes and close cursor and connection instances

In [ ]:
# Commit the changes to the database
db_conn.commit()

# Close the cursor and database connection
cursor.close()
db_conn.close()